# BUG: Beveridgean Unemployment Gap
This notebook demonstrates the python implementation of the "Beveridgean Unemployment Gap" by Pascal Michaillat and Emmanuel Saez. The original code was in MATLAB. See [GitHub link](https://github.com/pascalmichaillat/unemployment-gap) for original. 

## import packages

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import root

In [ ]:
import sys
sys.path.insert(0, '../bug')
import bug

## Read the data
Here, we read from the [excel file](https://github.com/pascalmichaillat/unemployment-gap/blob/main/code/data.xlsx) provided with the unemployment-gap matlab package.

The goal of this notebook is to re-create analysis and some figures from the Unemployment Gap paper, so that we can verify we are getting the *same* outputs. (*Sameness* allowing for some small differences between the two language implementations)

#### unemployment rate

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Unemployment rate (percent)', 'Year', 'Month'],)

In [ ]:
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
unempl_rate = pd.Series(data=df['Unemployment rate (percent)'].values,
                       index=dates, name='unempl_rate')

#### unemployment level

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Unemployment level (thousands of persons)', 'Year', 'Month'],)

In [ ]:
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
unempl_level = pd.Series(data=df['Unemployment level (thousands of persons)'].values,
                        index=dates, name='unempl_level')

#### short-term unemployment level

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Short-term unemployment level (thousands of persons)', 'Year', 'Month'],)

In [ ]:
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
ushort_level = pd.Series(data=df['Short-term unemployment level (thousands of persons)'].values, 
                        index=dates, name='short_term_unempl_level')

#### labor force level

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Labor force level (thousands of persons)', 'Year', 'Month'],)

In [ ]:
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
labor_level = pd.Series(data=df['Labor force level (thousands of persons)'].values,
                       index=dates, name='labor_force_level')

In [ ]:
find_rate =  bug.compute_job_finding_rate(unempl_level, ushort_level)

In [ ]:
find_rate.name='job_find_rate'

In [ ]:
find_rate

In [ ]:

def job_sep_eqn(lambda_rate, *consts):
    find_rate, u_level, u_level_next, h_level = consts
    expr = (1. - np.exp(-(find_rate+lambda_rate))) * lambda_rate/(find_rate+lambda_rate) * h_level + np.exp(-(find_rate+lambda_rate)) * u_level - u_level_next  
    
    return expr 


In [ ]:
solution = root(job_sep_eqn, 0, (find_rate, u_level, u_level_next, h_level))

In [ ]:
solution.x

In [ ]:
lambda_rate = [root(job_sep_eqn, 0, (find_rate.loc[t], unempl_level.loc[t], unempl_level.loc[t+1], labor_level.loc[t])).x[0] for t in find_rate.index[:-1]]    

In [ ]:
lambda_rate